In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the iexfinance api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [3]:
from iexfinance.stocks import get_historical_data
import iexfinance as iex
# dir(iex)
from iexfinance.refdata import get_symbols

# Data Collection

In this step, you will need to use the IEX api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [4]:
tickers = pd.DataFrame(get_symbols())
tickers.head()

symbol exchange                                 name        date type  \
0      A      NYS            Agilent Technologies Inc.  2020-01-21   cs   
1     AA      NYS                          Alcoa Corp.  2020-01-21   cs   
2   AAAU      PSE         Perth Mint Physical Gold ETF  2020-01-21   et   
3   AACG      NAS  ATA Creativity Global Sponsored ADR  2020-01-21   ad   
4   AADR      PSE  AdvisorShares Dorsey Wright ADR ETF  2020-01-21   et   

                  iexId region currency  isEnabled  
0  IEX_46574843354B2D52     US      USD       True  
1  IEX_4238333734532D52     US      USD       True  
2  IEX_474B433136332D52     US      USD       True  
3  IEX_44595A4C53392D52     US      USD       True  
4  IEX_5253355435362D52     US      USD       True

In [5]:
# COMMENTED OUT DUE TO OVERUSE OF DATA IN IEXFINANCE
# list_of_tickers = ["SPY", "AGG"]
# end_date = datetime.now()
# start_date = end_date + timedelta(-365)

# ticker_data = get_historical_data(list_of_tickers,start_date,end_date, output_format = "pandas")
# ticker_data.head()

# # ticker_data = ticker_data.drop(columns = ["open","high","low"],inplace = True)
# # ticker_data.head()

In [6]:
IEX_path = Path("../Resources/IEXCLOUD_data.csv")
ticker_data = pd.read_csv(IEX_path)
ticker_data["date"] =  pd.to_datetime(ticker_data.loc[:,"date"],infer_datetime_format = True)
ticker_data.set_index(ticker_data["date"],inplace = True)
ticker_data.drop(columns = ["date"], inplace = True)
ticker_data.tail()
# ticker_data.info()

SPY     AGG
date                      
2020-01-13  327.95  112.91
2020-01-14  327.45  113.01
2020-01-15  328.19  113.18
2020-01-16  330.92  113.18
2020-01-17  331.95  113.05

# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

In [7]:
# Calculate the daily roi for the stocks
# daily_returns = ticker_data.pct_change()["close"]
# daily_mean_SPY = daily_returns.mean()["SPY"]["close"]
# daily_mean_AGG = daily_returns.mean()["AGG"]["close"]

daily_returns = ticker_data.pct_change()
daily_mean_SPY = daily_returns.mean()["SPY"]
daily_mean_AGG = daily_returns.mean()["AGG"]

In [8]:
# Calculate volatility
volatility_SPY = ticker_data.std()["SPY"]
volatility_AGG = ticker_data.std()["AGG"]

In [9]:
# Save the last day's closing price
close_SPY = ticker_data["SPY"][-1]
close_AGG = ticker_data["AGG"][-1]
# close_AGG

In [10]:
# Setup the Monte Carlo Parameters
number_simulations = 500
num_trading_days = 252
number_records = 252 * 30
sim_price = pd.DataFrame()
monte_carlo = pd.DataFrame()

In [11]:
# Run the Monte Carlo Simulation
for x in range(number_simulations):
    
    simulated_SPY = [close_SPY]
    simulated_AGG = [close_AGG]
    
    for i in range(num_trading_days):
        
        simulated_SPY_prices = simulated_SPY[-1] * (1 + np.random.normal(daily_mean_SPY, volatility_SPY))
        simulated_AGG_prices = simulated_AGG[-1] * (1 + np.random.normal(daily_mean_AGG, volatility_AGG))
        
        simulated_SPY.append(simulated_SPY_prices)
        simulated_AGG.append(simulated_AGG_prices)
        
    
    sim_price["SPY_sim"] = pd.Series(simulated_SPY)
    sim_price["AGG_sim"] = pd.Series(simulated_AGG)
    
# monte_carlo.head()
    
    monte_carlo_DR = sim_price.pct_change()
    
#     # 60/40 portfolio using the SPY and AGG tickers to represent the 60% stocks (SPY) and 40% bonds (AGG).
    weights = [0.60, 0.40]

    portfolio_DR = monte_carlo_DR.dot(weights)
    
    monte_carlo[x] = (1 + portfolio_DR.fillna(0)).cumprod()
#     monte_carlo

monte_carlo.head()

0          1             2           3           4            5    \
0    1.000000   1.000000      1.000000    1.000000    1.000000     1.000000   
1    5.257274  -5.187505      9.772029   -7.035848   -3.170262     5.283059   
2   42.648406  26.035599    125.153616   -1.121904  -17.348137    20.900832   
3  -54.600675  17.371857   2338.174580  -16.755108  -30.945512   200.181255   
4 -853.959243  58.044837  38583.034150  106.194007 -365.416472  1053.770888   

           6            7           8          9    ...         490  \
0     1.000000     1.000000    1.000000   1.000000  ...    1.000000   
1   -15.855252   -16.126884  -11.760077   2.634785  ...   11.115404   
2   -49.037754  -256.702982   61.916121  -0.601857  ... -101.801277   
3   169.995483   268.960959  331.840311   3.849334  ...  -22.823158   
4 -3665.320077  5636.664179   57.232012 -60.117278  ...  141.460933   

          491         492          493         494        495         496  \
0    1.000000    1.000000     1.000000    1.000000   1.000000    1.000000   
1    7.502145    3.810528    -3.411789    4.067459  -6.967109    2.869967   
2  -36.548594   -1.939742   -44.205869   37.213429  -0.651758   13.248869   
3 -209.848308    9.575407  -451.132487   18.048685  -7.001917  -67.497387   
4 -412.248902 -204.679101 -1422.775331 -112.759548 -13.974544 -684.166378   

           497         498           499  
0     1.000000    1.000000      1.000000  
1   -21.338543    4.093804     10.168034  
2    92.850645   -2.374665    250.353333  
3  -775.598107  -20.469973  -2732.901628  
4  8895.878524 -612.630710  12411.353336  

[5 rows x 500 columns]

In [ ]:
# Visualize the Simulation
# YOUR CODE HERE

In [14]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
# YOUR CODE HERE

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 20 years)
# YOUR CODE HERE

In [ ]:
# Display the 90% confidence interval for the ending returns
# YOUR CODE HERE

In [ ]:
# Visualize the distribution of the ending returns
# YOUR CODE HERE

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
# YOUR CODE HERE

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# YOUR CODE HERE

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# YOUR CODE HERE